In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Get the data path
data_path = Path.cwd().parent.resolve()/ "src" / "hidden_debt_gsf" /"data"/"GFSISB"/"GFSIBS2020"

# Define the file name
file_name = "GFSIBS2020_01-16-2025.csv"

# Combine the base path and file name
file_path = data_path / file_name

# Read the file in chunks
chunksize = 10_000

if file_path.exists():  # Check if the file exists
    data_chunks = pd.read_csv(file_path, chunksize=chunksize)
    data = pd.concat(data_chunks, ignore_index=True)
    print("File loaded successfully.")
else:
    print(f"File not found at {file_path}")


/tmp/ipykernel_125025/3832726373.py:15: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(data_chunks, ignore_index=True)
/tmp/ipykernel_125025/3832726373.py:15: DtypeWarning: Columns (59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(data_chunks, ignore_index=True)
/tmp/ipykernel_125025/3832726373.py:15: DtypeWarning: Columns (62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(data_chunks, ignore_index=True)
/tmp/ipykernel_125025/3832726373.py:15: DtypeWarning: Columns (62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(data_chunks, ignore_index=True)
/tmp/ipykernel_125025/3832726373.py:15: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(data_chunks, ignore_index=True)
/tmp/ipykernel_125025/383272

File loaded successfully.


In [13]:
print(len(data))
var_of_interest = data[data['Stocks, Transactions, and Other Flows Name']=='Net incurrence of liabilities']
print(len(var_of_interest))
var_of_interest = var_of_interest[var_of_interest['Sector Name']=='Budgetary central government']
print(len(var_of_interest))
var_of_interest = var_of_interest[var_of_interest['Unit Name']=='Domestic currency']
print(len(var_of_interest))
var_of_interest = var_of_interest[var_of_interest['Residence Name']=='Total']
print(len(var_of_interest))
var_of_interest = var_of_interest[var_of_interest['Instrument and Assets Classification Name']=='Total financial assets/liabilities ']
print(len(var_of_interest))
var_of_interest = var_of_interest[var_of_interest['Attribute']=='Value']
print(len(var_of_interest))
var_of_interest.to_csv('var_of_interest.csv', index=False)


3374302
488187
64160
32119
14487
1054
160


In [5]:
# Extract headers and save to CSV
headers = pd.DataFrame(data.columns, columns=["Headers"])
headers.to_csv("headers.csv", index=False)
print(f"Headers saved")

# Columns to extract unique values from
columns_to_extract = ["Residence Name", "Instrument and Assets Classification Name"]

# Extract and save unique values for each column
for column in columns_to_extract:
    unique_values = data[column].dropna().unique()  # Drop NaN values and get unique ones
    unique_df = pd.DataFrame(unique_values, columns=[column])  # Convert to DataFrame
    output_file = f"unique_{column.replace(' ', '_')}.csv"  # Save file
    unique_df.to_csv(output_file, index=False)
    print(f"Unique values from '{column}' saved to {output_file}")

Headers saved
Unique values from 'Residence Name' saved to unique_Residence_Name.csv
Unique values from 'Instrument and Assets Classification Name' saved to unique_Instrument_and_Assets_Classification_Name.csv


In [4]:
def filter_and_combine(data, column_name, keywords, output_filename):
    """
    Filters rows based on multiple keywords in a specified column and combines them into one CSV.
    
    Parameters:
        data (pd.DataFrame): The DataFrame to search within.
        column_name (str): The column to search for the keywords.
        keywords (list): A list of keywords to search for.
        output_filename (str): The filename to save the combined filtered rows.

    Returns:
        pd.DataFrame: Combined DataFrame of filtered rows.
    """
    combined_rows = pd.DataFrame()  # Initialize an empty DataFrame

    for keyword in keywords:
        # Filter rows where the keyword appears in the specified column (case-insensitive)
        filtered_rows = data[data[column_name].str.contains(keyword, case=False, na=False)]
        
        # Add a column to indicate the keyword used for filtering
        filtered_rows['Keyword'] = keyword
        
        # Append to the combined DataFrame
        combined_rows = pd.concat([combined_rows, filtered_rows], ignore_index=True)

    # Save the combined rows to a single CSV file
    # combined_rows.to_csv(Path.cwd() / output_filename, index=False)
    
    # Return the combined DataFrame
    return combined_rows

# Define the DataFrame (assuming `data` is already loaded)
column_name = "Stocks, Transactions, and Other Flows Name"

# List of keywords to search for
keywords = ["debt", "liabilities", "borrowing"]

# Call the function to filter and combine rows
output_filename = "filtered_rows_combined.csv"
combined_data = filter_and_combine(data, column_name, keywords, output_filename)

# Display the unique values in the "Classification Name" column
unique_classification_names = combined_data[column_name].unique()
print("Unique classification names:", unique_classification_names)

/tmp/ipykernel_83409/63332209.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword


Unique classification names: ['Holding gains and losses in liabilities' 'Net incurrence of liabilities'
 'Stock position liabilities' 'Volume changes in liabilities'
 'Other economic flows in liabilities']


In [ ]:
def analyze_data_format(data):
    """
    Analyzes the dataset to calculate the sum of legitimate entries and covered countries
    for each combination of Classification, Sector, and Unit.
    
    Parameters:
        data (pd.DataFrame): The input dataset.

    Returns:
        pd.DataFrame: Summary DataFrame with analysis results.
    """
    # Filter rows where Attribute is "Value"
    filtered_data = data[data['Attribute'] == 'Value']

    # Select year columns (assume all columns except metadata and Attribute are years)
    year_columns = [col for col in data.columns if col.startswith('20') or col.startswith('19')]

    # Group by Classification, Sector, and Unit
    grouped = filtered_data.groupby(
        ['Stocks, Transactions, and Other Flows Name', 'Sector Name', 'Unit Name', 'Residence Name', 'Instrument and Assets Classification Name']
    )

    # Calculate the required summaries
    summary = grouped.apply(
        lambda group: pd.Series({
            'Sum of Legitimate Entries': group[year_columns].apply(
                pd.to_numeric, errors='coerce'
            ).count().sum(),
            'Number of Covered Countries': group['Country Code'].nunique()
        })
    ).reset_index()

    return summary

# Assuming your dataset is loaded in `data`
summary = analyze_data_format(combined_data)

# Save the summary to a CSV file if needed
summary.to_csv(Path.cwd().parent.resolve()/"data"/"GFSISB"/"Summaries"/"summary_analysis_2015.csv", index=False)


  Stocks, Transactions, and Other Flows Name Sector Name Unit Name  \
0                                     Stocks    Sector B       EUR   
1                               Transactions    Sector A       USD   

  Residence Name Instrument and Assets Classification Name  \
0        Foreign                          Classification Y   
1       Domestic                          Classification X   

   Sum of Legitimate Entries  Number of Covered Countries  
0                          4                            2  
1                          4                            2  


/tmp/ipykernel_114180/3917077512.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = grouped.apply(


In [1]:
import pandas as pd
from pathlib import Path

# Define the data directory and output directory
data_dir = Path.cwd().parent.resolve()/ "src" / "hidden_debt_gsf" / "data" / "GFSISB"
output_dir = data_dir / "Summaries"

output_dir.mkdir(parents=True, exist_ok=True)  # Ensure the output directory exists

# List of years to process
# years = [2014, 2015, 2016, 2017, 2019, 2020]

# List of years to process, to attain varnames
years = [2014]

# List of keywords for filtering
keywords = ["debt", "liabilities", "borrowing"]

# Chunksize for reading large files
chunksize = 10_000

def load_data(data_dir, year, chunksize):
    """Load the CSV file for a specific year in chunks."""
    file_dir = f"GFSIBS{year}"
    file_name = f"GFSIBS{year}_01-16-2025.csv"
    file_path = data_dir / file_dir / file_name
    if file_path.exists():
        data_chunks = pd.read_csv(file_path, chunksize=chunksize)
        return pd.concat(data_chunks, ignore_index=True)
    else:
        print(f"File not found for year {year}: {file_path}")
        return None

def filter_and_combine(data, column_name, keywords):
    """Filter rows based on multiple keywords and return combined results."""
    combined_rows = pd.DataFrame()
    for keyword in keywords:
        filtered_rows = data[data[column_name].str.contains(keyword, case=False, na=False)]
        filtered_rows['Keyword'] = keyword
        combined_rows = pd.concat([combined_rows, filtered_rows], ignore_index=True)
    return combined_rows

def analyze_data_format(data):
    """Analyze data to calculate sums of legitimate entries and covered countries."""
    filtered_data = data[data['Attribute'] == 'Value']
    year_columns = [col for col in data.columns if col.startswith(('20', '19'))]
    grouped = filtered_data.groupby(
        ['Stocks, Transactions, and Other Flows Name', 'Sector Name', 'Unit Name', 
         'Residence Name', 'Instrument and Assets Classification Name']
    )
    summary = grouped.apply(
        lambda group: pd.Series({
            'Sum of Legitimate Entries': group[year_columns].apply(pd.to_numeric, errors='coerce').count().sum(),
            'Number of Covered Countries': group['Country Code'].nunique()
        })
    ).reset_index()
    return summary

def process_all_years(data_dir, years, keywords, output_dir, chunksize):
    """Process all years, filter and analyze data, and save summaries."""
    for year in years:
        # Load data for the year
        data = load_data(data_dir, year, chunksize)
        if data is None:
            continue  # Skip if file not found

        # Filter and combine rows based on keywords
        combined_data = filter_and_combine(data, "Stocks, Transactions, and Other Flows Name", keywords)

        # Analyze data format
        summary = analyze_data_format(combined_data)

        # Save summary to a CSV file
        summary_file = output_dir / f"summary_analysis_{year}.csv"
        summary.to_csv(summary_file, index=False)
        print(f"Summary for {year} saved to {summary_file}")

# Execute the processing for all years
process_all_years(data_dir, years, keywords, output_dir, chunksize)

/tmp/ipykernel_108934/3284056054.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword


Summary for 2014 saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSISB/Summaries/summary_analysis_2014.csv


/tmp/ipykernel_108934/3284056054.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = grouped.apply(


In [2]:
import pandas as pd
from pathlib import Path

# Define the directory containing summary files
summary_dir = output_dir

# List of years to process
years = [2014, 2015, 2016, 2017, 2019, 2020]

# Load all summary files into a list of DataFrames
summary_dfs = []
for year in years:
    file_path = summary_dir / f"summary_analysis_{year}.csv"
    if file_path.exists():
        summary_dfs.append(pd.read_csv(file_path))
    else:
        print(f"Summary file for {year} not found at {file_path}")

# Combine all summaries into a single DataFrame
combined_summary = pd.concat(summary_dfs, ignore_index=True)

# Group by relevant columns and calculate aggregated values
aggregated_summary = combined_summary.groupby(
    # ['Stocks, Transactions, and Other Flows Name', 'Sector Name', 'Unit Name'],
    ['Stocks, Transactions, and Other Flows Name', 'Sector Name', 'Unit Name', 'Residence Name', 'Instrument and Assets Classification Name'],
    as_index=False
).agg({
    'Sum of Legitimate Entries': 'sum',
    'Number of Covered Countries': 'max'
})

# Sort by the most legitimate entries
aggregated_summary = aggregated_summary.sort_values(by='Sum of Legitimate Entries', ascending=False)

# Save the aggregated summary to a CSV file
output_path = summary_dir / "aggregated_summary_2014_2020.csv"
aggregated_summary.to_csv(output_path, index=False)

print(f"Aggregated summary saved to {output_path}")


Aggregated summary saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSISB/Summaries/aggregated_summary_2014_2020.csv


In [14]:
import pandas as pd
from pathlib import Path

# Define the data directory and output directory
data_dir = Path.cwd().parent.resolve() / "data" / "GFSISB"
output_dir = data_dir / "Summaries"
output_dir.mkdir(parents=True, exist_ok=True)  # Ensure the output directory exists

# List of years to process
years = [2014, 2015, 2016, 2017, 2019, 2020]

# Chunksize for reading large files
chunksize = 5_000

# Variables to process
variables_to_process = [
    {
        "Stocks, Transactions, and Other Flows Name": "Net incurrence of liabilities",
        "Sector Name": "Budgetary central government",
        "Unit Name": "Domestic currency",
        "Residence Name": "External",
        "Instrument and Assets Classification Name": "Total financial assets/liabilities ",
    },
    {
        "Stocks, Transactions, and Other Flows Name": "Net incurrence of liabilities",
        "Sector Name": "Budgetary central government",
        "Unit Name": "Domestic currency",
        "Residence Name": "Total",
        "Instrument and Assets Classification Name": "Total financial assets/liabilities ",
    },
    {
        "Stocks, Transactions, and Other Flows Name": "Net incurrence of liabilities",
        "Sector Name": "Budgetary central government",
        "Unit Name": "Domestic currency",
        "Residence Name": "Domestic",
        "Instrument and Assets Classification Name": "Total financial assets/liabilities ",
    },
]

def load_data(data_dir, year, chunksize):
    """Load the CSV file for a specific year in chunks."""
    file_dir = f"GFSIBS{year}"
    file_name = f"GFSIBS{year}_01-16-2025.csv"
    file_path = data_dir / file_dir / file_name
    if file_path.exists():
        data_chunks = pd.read_csv(file_path, chunksize=chunksize)
        return pd.concat(data_chunks, ignore_index=True)
    else:
        print(f"File not found for year {year}: {file_path}")
        return None

def calculate_numeric_counts(data, variables_to_process, year):
    """Calculate the number of numeric entries for each year column."""
    year_columns = [col for col in data.columns if col.startswith(('20', '19'))]
    print(f"Processing year {year} with year columns: {year_columns}")
    results = []

    for variable in variables_to_process:
        print(f"Processing variable: {variable}")
        filtered_data = data.copy()

        for col, value in variable.items():
            print(f"Filtering for column: {col} with value: {value}")
            if col not in filtered_data.columns:
                print(f"Column {col} not found in data!")
                continue
            print(f"Unique values in column {col}: {filtered_data[col].unique()}")
            filtered_data = filtered_data[filtered_data[col] == value]

        if filtered_data.empty:
            print(f"No data found for variable: {variable} in year {year}")
            continue

        # Count numeric entries in year columns
        counts = filtered_data[year_columns].apply(pd.to_numeric, errors='coerce').count()
        result = {"Year": year}
        result.update(counts.to_dict())
        results.append(result)

    return results


def process_variables_by_year(data_dir, years, variables_to_process, output_dir, chunksize):
    """Process variables by year and save results as individual DataFrames."""
    results_per_variable = {i: [] for i in range(len(variables_to_process))}

    for year in years:
        # Load data for the year
        data = load_data(data_dir, year, chunksize)
        if data is None:
            continue  # Skip if file not found

        # Calculate numeric counts for each variable
        year_results = calculate_numeric_counts(data, variables_to_process, year)

        # Append results for each variable
        for i, result in enumerate(year_results):
            results_per_variable[i].append(result)

    # Save results for each variable as a DataFrame
    for i, variable_results in results_per_variable.items():
        if not variable_results:  # Skip if no results for a variable
            print(f"No data found for variable {i + 1}.")
            continue

        df = pd.DataFrame(variable_results)
        # Ensure the Year column exists
        if "Year" not in df.columns:
            df["Year"] = []

        # Reorder columns (Year first, then year columns in ascending order)
        year_columns = sorted([col for col in df.columns if col != "Year"])
        df = df[["Year"] + year_columns]

        # Save to CSV
        output_file = output_dir / f"variable_{i + 1}_numeric_counts.csv"
        df.to_csv(output_file, index=False)
        print(f"Results for variable {i + 1} saved to {output_file}")

# Execute the processing
process_variables_by_year(data_dir, years, variables_to_process, output_dir, chunksize)


: 